In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
churn = pd.read_csv('C:/Users/acerpc/Desktop/data_folder/telecom_churn.csv')
churn.head()

,subscriberID,churn,gender,AGE,edu_class,incomeCode,duration,feton,peakMinAv,peakMinDiff,posTrend,negTrend,nrProm,prom,curPlan,avgplan,planChange,posPlanChange,negPlanChange,call_10086
0,19164958.0,1.0,0.0,20.0,2.0,12.0,16.0,0.0,113.666667,-8.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,39244924.0,1.0,1.0,20.0,0.0,21.0,5.0,0.0,274.000000,-371.0,0.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,0.0,1.0
2,39578413.0,1.0,0.0,11.0,1.0,47.0,3.0,0.0,392.000000,-784.0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,1.0
3,40992265.0,1.0,0.0,43.0,0.0,4.0,12.0,0.0,31.000000,-76.0,0.0,1.0,2.0,1.0,3.0,3.0,0.0,0.0,0.0,1.0
4,43061957.0,1.0,1.0,60.0,0.0,9.0,14.0,0.0,129.333333,-334.0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
x = churn.loc[:,'gender':]
y = churn['churn']
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.3,random_state=0)

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler_x_train = scaler.fit_transform(train_x)
scaler_x_test = scaler.transform(test_x)

In [8]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(50,50),solver='sgd',max_iter=10,learning_rate_init=0.1,random_state=1)
mlp.fit(scaler_x_train,train_y)

D:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 50), learning_rate='constant',
       learning_rate_init=0.1, max_iter=10, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [19]:
predict_train = mlp.predict(scaler_x_train)
predict_test = mlp.predict(scaler_x_test)
proba_train = mlp.predict_proba(scaler_x_train)[:,1]
proba_test = mlp.predict_proba(scaler_x_test)[:,1]

In [18]:
proba_test.shape
test_y.shape

(2424,)

In [20]:
from sklearn import metrics

print(metrics.roc_auc_score(test_y,proba_test))

0.9154222489838928


In [21]:
print(metrics.classification_report(test_y,predict_test))
print(metrics.confusion_matrix(test_y,predict_test))

              precision    recall  f1-score   support

         0.0       0.69      0.98      0.81       584
         1.0       0.94      0.43      0.59       455

   micro avg       0.74      0.74      0.74      1039
   macro avg       0.82      0.71      0.70      1039
weighted avg       0.80      0.74      0.71      1039

[[572  12]
 [258 197]]


In [22]:
mlp.score(scaler_x_test,test_y)

0.7401347449470644

In [25]:
from sklearn.model_selection import GridSearchCV

mlp1 = MLPClassifier(solver='sgd')
param_grid = {'hidden_layer_sizes':[(i,j) for i in range(101) for j in range(101)],'max_iter':list(range(10,1010,10)),
              'learning_rate_init':list(np.arrange(0.001,0.1,0.001))}
cv = GridSearchCV(mlp1,param_grid,scoring='roc_auc',cv=5,n_jobs=-1)

TypeError: 'float' object cannot be interpreted as an integer

1. hidden_layer_sizes :元祖格式，长度=n_layers-2, 默认(100，），第i个元素表示第i个隐藏层的神经元的个数。 
2. activation :{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, 默认‘relu 
- ‘identity’： no-op activation, useful to implement linear bottleneck， 
返回f(x) = x 
- ‘logistic’：the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)). 
- ‘tanh’：the hyperbolic tan function, returns f(x) = tanh(x). 
- ‘relu’：the rectified linear unit function, returns f(x) = max(0, x) 
4. solver： {‘lbfgs’, ‘sgd’, ‘adam’}, 默认 ‘adam’，用来优化权重 
- lbfgs：quasi-Newton方法的优化器 
- sgd：随机梯度下降 
- adam： Kingma, Diederik, and Jimmy Ba提出的机遇随机梯度的优化器 
注意：默认solver ‘adam’在相对较大的数据集上效果比较好（几千个样本或者更多），对小数据集来说，lbfgs收敛更快效果也更好。 
5. alpha :float,可选的，默认0.0001,正则化项参数 
6. batch_size : int , 可选的，默认‘auto’,随机优化的minibatches的大小，如果solver是‘lbfgs’，分类器将不使用minibatch，当设置成‘auto’，batch_size=min(200,n_samples) 
7. learning_rate :{‘constant’，‘invscaling’, ‘adaptive’},默认‘constant’，用于权重更新，只有当solver为’sgd‘时使用 
- ‘constant’: 有‘learning_rate_init’给定的恒定学习率 
- ‘incscaling’：随着时间t使用’power_t’的逆标度指数不断降低学习率learning_rate_ ，effective_learning_rate = learning_rate_init / pow(t, power_t) 
- ‘adaptive’：只要训练损耗在下降，就保持学习率为’learning_rate_init’不变，当连续两次不能降低训练损耗或验证分数停止升高至少tol时，将当前学习率除以5. 
8. max_iter: int，可选，默认200，最大迭代次数。 
9. random_state:int 或RandomState，可选，默认None，随机数生成器的状态或种子。 
10. shuffle: bool，可选，默认True,只有当solver=’sgd’或者‘adam’时使用，判断是否在每次迭代时对样本进行清洗。 
11. tol：float, 可选，默认1e-4，优化的容忍度 
12. learning_rate_int:double,可选，默认0.001，初始学习率，控制更新权重的补偿，只有当solver=’sgd’ 或’adam’时使用。 
13. power_t: double, optional, default 0.5，只有solver=’sgd’时使用，是逆扩展学习率的指数.当learning_rate=’invscaling’，用来更新有效学习率。 
14. verbose : bool, optional, default False,是否将过程打印到stdout 
15. warm_start : bool, optional, default False,当设置成True，使用之前的解决方法作为初始拟合，否则释放之前的解决方法。 
16. momentum : float, default 0.9,Momentum(动量） for gradient descent update. Should be between 0 and 1. Only used when solver=’sgd’. 
17. nesterovs_momentum : boolean, default True, Whether to use Nesterov’s momentum. Only used when solver=’sgd’ and momentum > 0. 
18. early_stopping : bool, default False,Only effective when solver=’sgd’ or ‘adam’,判断当验证效果不再改善的时候是否终止训练，当为True时，自动选出10%的训练数据用于验证并在两步连续爹迭代改善低于tol时终止训练。 
19. validation_fraction : float, optional, default 0.1,用作早期停止验证的预留训练数据集的比例，早0-1之间，只当early_stopping=True有用 
20. beta_1 : float, optional, default 0.9，Only used when solver=’adam’，估计一阶矩向量的指数衰减速率，[0,1)之间 
21. beta_2 : float, optional, default 0.999,Only used when solver=’adam’估计二阶矩向量的指数衰减速率[0,1)之间 
22. epsilon : float, optional, default 1e-8,Only used when solver=’adam’数值稳定值。 
属性说明： 
- classes_:每个输出的类标签 
- loss_:损失函数计算出来的当前损失值 
- coefs_:列表中的第i个元素表示i层的权重矩阵 
- intercepts_:列表中第i个元素代表i+1层的偏差向量 
- n_iter_ ：迭代次数 
- n_layers_:层数 
- n_outputs_:输出的个数 
- out_activation_:输出激活函数的名称。 
方法说明： 
- fit(X,y):拟合 
- get_params([deep]):获取参数 
- predict(X):使用MLP进行预测 
- predic_log_proba(X):返回对数概率估计 
- predic_proba(X)：概率估计 
- score(X,y[,sample_weight]):返回给定测试数据和标签上的平均准确度 
-set_params(**params):设置参数。

